In [188]:
import pandas as pd
import numpy as np
import re
import datetime as dt

# Importar bases
Delitos sexuales (ds), homicidios (homi) y vilencia intrafamiliar (VIF)

In [189]:
ds = pd.read_excel("ds.xlsx")
homi = pd.read_excel("homicidios.xlsx")
vif = pd.read_csv("vif.csv")

/tmp/ipykernel_389/1727000379.py:3: DtypeWarning: Columns (13,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  vif = pd.read_csv("vif.csv")


# Procesamiento VIF

- Remover el los registros sin cedula
- Contar el número de casos por víctima teniendo encuenta el id de denuncia
- Extraer en una base la primera o unica denuncia por víctima 
- Imputar datos en caso de registros faltantes si una persona tiene multiples registros

In [190]:
## Remover el los registros sin cedula
vif2 = vif.dropna(subset=['numero_documento'])
print(vif.shape)
print(vif2.shape)

(77964, 39)
(73302, 39)


In [191]:
vif2.columns

Index(['Unnamed: 0', 'arma_medio', 'barrio_hecho', 'barrio_residencia', 'cai',
       'codigo_barrio_hecho', 'codigo_barrio_residencia',
       'codigo_comuna_hecho', 'codigo_comuna_residencia', 'comuna_hecho',
       'comuna_residencia', 'conducta', 'conducta_especial',
       'consecutivo_theta', 'cuadrante_hecho', 'departamento_policia',
       'direccion', 'edad', 'estacion', 'estado_civil', 'fecha_denuncia',
       'fecha_hecho', 'lugar', 'modalidad_1257', 'modalidad_294',
       'nivel_academico', 'nombre_completo', 'numero_documento', 'ocupacion',
       'otra_relacion_denunciado_victima', 'parentesco', 'punto_geografico',
       'punto_plano', 'sede_receptora', 'sexo', 'telefono', 'testigo',
       'zona_hecho', 'zona_residencia'],
      dtype='object')

In [192]:
## Contar el número de casos por víctima teniendo encuenta el id de denuncia
casos_vif = vif2.drop_duplicates(subset=['numero_documento', 'consecutivo_theta'], keep= 'last').groupby('numero_documento', as_index=False)['Unnamed: 0'].count()
casos_vif.rename(columns={"Unnamed: 0": "reincidencia"}, inplace=True)

In [193]:
## Extraer en una base la primera o unica denuncia por víctima
primer_caso = vif2.sort_values('fecha_denuncia').drop_duplicates(subset=['numero_documento'], keep= 'first')
primer_caso.columns

Index(['Unnamed: 0', 'arma_medio', 'barrio_hecho', 'barrio_residencia', 'cai',
       'codigo_barrio_hecho', 'codigo_barrio_residencia',
       'codigo_comuna_hecho', 'codigo_comuna_residencia', 'comuna_hecho',
       'comuna_residencia', 'conducta', 'conducta_especial',
       'consecutivo_theta', 'cuadrante_hecho', 'departamento_policia',
       'direccion', 'edad', 'estacion', 'estado_civil', 'fecha_denuncia',
       'fecha_hecho', 'lugar', 'modalidad_1257', 'modalidad_294',
       'nivel_academico', 'nombre_completo', 'numero_documento', 'ocupacion',
       'otra_relacion_denunciado_victima', 'parentesco', 'punto_geografico',
       'punto_plano', 'sede_receptora', 'sexo', 'telefono', 'testigo',
       'zona_hecho', 'zona_residencia'],
      dtype='object')

In [194]:
primer_caso = primer_caso.merge(casos_vif, on='numero_documento', how='left')

In [195]:
print(primer_caso[["parentesco", "ocupacion", "edad", "testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"]].isna().sum())
print(primer_caso.shape)

parentesco         50192
ocupacion          17305
edad                   0
testigo                0
estado_civil         439
arma_medio         57117
modalidad_1257      4472
modalidad_294       4472
nivel_academico      541
sexo                2404
dtype: int64
(60631, 40)


In [196]:
##- Imputar datos en caso de registros faltantes si una persona tiene multiples registros
#intentamos usar la segunda denuncia para la imputacion
casos_vif_segunda = pd.merge(vif2, casos_vif[casos_vif["reincidencia"]>1] , how ='inner', on =['numero_documento']).sort_values('fecha_denuncia').groupby('numero_documento', as_index=False)["numero_documento","fecha_denuncia","parentesco", "ocupacion", "edad", "testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"].nth(1)

/tmp/ipykernel_389/1701631397.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  casos_vif_segunda = pd.merge(vif2, casos_vif[casos_vif["reincidencia"]>1] , how ='inner', on =['numero_documento']).sort_values('fecha_denuncia').groupby('numero_documento', as_index=False)["numero_documento","fecha_denuncia","parentesco", "ocupacion", "edad", "testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"].nth(1)


In [197]:
#primer_caso[["numero_documento","fecha_denuncia","parentesco", "ocupacion", "edad", "testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"]]

primer_caso_new = pd.merge(primer_caso, casos_vif_segunda, how="left",  on =['numero_documento'])
primer_caso_new.columns
primer_caso_new['arma_medio'] = np.where(primer_caso_new['arma_medio_x'].isna(), "Sin arma",primer_caso_new['arma_medio_x'])
primer_caso_new['estado_civil'] = np.where(primer_caso_new['estado_civil_x'].isna(), primer_caso_new['estado_civil_y'],primer_caso_new['arma_medio_x'])
primer_caso_new['sexo'] = np.where(primer_caso_new['sexo_x'].isna(), primer_caso_new['sexo_y'],primer_caso_new['sexo_x'])
primer_caso_new['modalidad_1257'] = np.where(primer_caso_new['modalidad_1257_x'].isna(), primer_caso_new['modalidad_1257_y'],primer_caso_new['modalidad_1257_x'])
primer_caso_new['ocupacion'] = np.where(primer_caso_new['ocupacion_x'].isna(), primer_caso_new['ocupacion_y'],primer_caso_new['ocupacion_x'])
primer_caso_new['nivel_academico'] = np.where(primer_caso_new['nivel_academico_x'].isna(), primer_caso_new['nivel_academico_y'],primer_caso_new['nivel_academico_x'])
primer_caso_new['modalidad_294'] = np.where(primer_caso_new['modalidad_294_x'].isna(), primer_caso_new['modalidad_294_y'],primer_caso_new['modalidad_294_x'])

print(primer_caso_new[["parentesco_x", "ocupacion", "edad_x", "testigo_x", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"]].isna().sum())

parentesco_x       50192
ocupacion          15995
edad_x                 0
testigo_x              0
estado_civil       57112
arma_medio             0
modalidad_1257      3889
modalidad_294       3889
nivel_academico      480
sexo                2004
dtype: int64


In [198]:
##subbase con las variables viables a trabajar
primer_caso_new = primer_caso_new[["numero_documento","fecha_denuncia_x","ocupacion", "edad_x", "testigo_x", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo", "reincidencia"]]
primer_caso_new.rename(columns={"fecha_denuncia_x": "fecha_denuncia", "edad_x": "edad", "testigo_x": "testigo"}, inplace=True)
print(primer_caso_new.columns)

Index(['numero_documento', 'fecha_denuncia', 'ocupacion', 'edad', 'testigo',
       'arma_medio', 'modalidad_1257', 'modalidad_294', 'nivel_academico',
       'sexo', 'reincidencia'],
      dtype='object')


In [199]:
primer_caso_new["fecha_denuncia"]

0        2004-10-26
1        2004-10-26
2        2004-10-27
3        2004-10-27
4        2004-10-29
            ...    
60626    2019-09-27
60627    2019-09-27
60628    2019-09-27
60629    2019-09-27
60630    2019-09-27
Name: fecha_denuncia, Length: 60631, dtype: object

In [200]:
primer_caso_new["fecha_denuncia"] =pd.to_datetime(primer_caso_new["fecha_denuncia"], format= '%Y-%m-%d')

# Procesamiento ds y homicidio
- Remover el los registros sin cedula
- Cambiar el formato de la varible "numero_documento"

In [201]:
homi = homi.dropna(subset=['numero_documento'])
homi["numero_documento"]= homi["numero_documento"].str.extract(pat='(\d+)', expand=False).astype(int)
ds = ds.dropna(subset=['numero_documento'])
ds["numero_documento"]= ds["numero_documento"].str.extract(pat='(\d+)', expand=False).astype(int)


# Union de las bases de homicidio y ds con vif
- Union de ds con vif teniendo en cuenta que solo contar los registros que tienen 7 dias o más de diferencia y que la base de vif puede tener id repetidos.
- Union de homicidio con vif

In [202]:
#union con ds
print(primer_caso_new.shape)
primer_caso = pd.merge(primer_caso_new, ds, how="left",  on =['numero_documento'])
primer_caso["incluir_ds"] = np.where(abs(((pd.to_datetime(primer_caso['fecha_denuncia_y'])- primer_caso['fecha_denuncia_x']).dt.days)<=7)| primer_caso['fecha_denuncia_y'].isna() , 0, 1)
primer_caso = primer_caso.groupby("numero_documento", as_index=False)["incluir_ds"].sum()
primer_caso_new = pd.merge(primer_caso_new, primer_caso, how="left",  on =['numero_documento'])
primer_caso_new.shape

(60631, 11)


(60631, 12)

In [203]:
primer_caso.groupby("incluir_ds")["incluir_ds"].count()

incluir_ds
0    60418
1      206
2        6
3        1
Name: incluir_ds, dtype: int64

In [204]:
#union con homi
primer_caso_new = pd.merge(primer_caso_new, homi[["numero_documento", "Index"]], how="left",  on =['numero_documento'])
primer_caso_new.shape

(60631, 13)

In [205]:
primer_caso_new.columns

Index(['numero_documento', 'fecha_denuncia', 'ocupacion', 'edad', 'testigo',
       'arma_medio', 'modalidad_1257', 'modalidad_294', 'nivel_academico',
       'sexo', 'reincidencia', 'incluir_ds', 'Index'],
      dtype='object')

In [206]:
#base final
primer_caso_new["reincidente"] = np.where((primer_caso_new['incluir_ds'] <=0) & (primer_caso_new['reincidencia']<=1) & (primer_caso_new['Index'].isna()), 0, 1)

In [207]:
primer_caso_new.groupby("reincidente")["reincidente"].count()

reincidente
0    52054
1     8577
Name: reincidente, dtype: int64

60631